## Honors College IHT Capstone | Example-Based Natural Language Generation with Context-Free Phrase Structure Grammars to Analyze Poverty of Stimulus Arguments
#### Author: Brian Brown | May 2020 | brianbrown0417@gmail.com

### Motivation
I was motivated to start this project after reading The Language Instinct by Steven Pinker. I was fascinated by the idea that humans possess a biological language faculty as a product of evolution. 

In my undergraduate coursework, I looked deeper into surrounding theories, such as the argument from the poverty of stimulus. In short, this is the idea that children are not exposed to rich enough data within their linguistic environments to acquire every feature of their language; therefore, they must posses some knowledge innately.

I also researched compilers, relying heavily on context-free grammars and different parsing algorithms (LL1, CKY, etc.). This knowledge paired well with my existing linguistic knowledge. However, I wanted to get serious, so I read Syntactic Structures by Noam Chomsky. This helped me understand how grammars could be represented in phrase structures, and how simple transformations create the possibility for an almost infinite number of possibilities.

### Goals
The aim of this notebook is to explore some of the fundamental linguistic tools and how they can be served to construct a simple, albeit effective natural langauge processing system. There were a lot of useful references, but I found https://www.thoughtco.com/phrase-structure-grammar-1691509 to be especially useful.

The first fundamental aspect is how context free-grammars can be used to describe consituent rules in English. On top of that, feature structures to ensure grammatical and semantic concord can be utilize to augment the context free grammar. Finally, some simple logic can go a long way in allowing the system to perform specific tasks.

The uasage of formal grammar rules show the potential of phrase structure grammars. In addition, the results can help inform our view of the poverty of stimulus argument. Showing the capabilities of a system with this strucutre, and the sentences it is not only able to parse, but also generate, is a testament to the robustness of the poverty of stimulus argument. This notebook does not rely on rigorous training, which offers an interesting tradoff. While this is a high maintenace solution. It is highly structured, modular, and permits iterability. These new iterations can be viewed as simulating language acquisition from stimulus. 

Nonetheless, the debate surrounding the poverty of stimulus are broad and contreversial. While the results of this notebook barely scratch the surface of, it is enough to provoke thought on the topic, and gain a unique, functional persepctive through computer science.

## Part 1: Constructing the Grammar
Part 1 aims to construct a grammatical structure that represents the English Language.

### A: Grammar Class

The following class is responsible for holding all structures associated with the grammar. It also includes some other simple object oriented functions to facilitate object-oriented functionality. This can be thought of as the cornerstone of this entire notebook. It holds the very basic rules that make up the grammar, and it does so with a very simple list of of rules.

I should also note that nltk proved to be very useful here. I will talk more about the feature stucture library; however, the functionality that is available after one import statement is astonishing.

In [93]:
from tabulate import tabulate
import random
import nltk

class Grammar(object):
    
    #basic constructure that initializes an empty dictionary to store rules
    def __init__(self):      
        self.rules = []
        
        
    def add_rule(self, feature_structure):
        
        for i in self.rules:
            if i == feature_structure:
                return
            
        self.rules.append(feature_structure)     
    
    
    def apply_rules(self, value):
        
        tags = value.split(' ')
        
        if len(tags) == 1:
            
            for i in self.rules:   
                if i['parent']['terminal'] == 'true' and i['child1']['tag'] == tags[0]:  
                    return i['parent']
                
        elif len(tags) == 2:
                        
            for i in self.rules:
                if i['parent']['terminal'] == 'false' and i['child1']['tag'] == tags[0] and i['child2']['tag'] == tags[1]: 
                    return i['parent']
        else:
            
            return None
    
    
    def get_rules(self, key):
        
        feature_structure_list = []
        
        for i in self.rules:
            if i['parent']['tag'] == key:
                feature_structure_list.append(i)
                
        return feature_structure_list
    
    
    def print_expansion(self, production):
        if production.get_left() is None or production.get_right() is None:
            return production.get_text()
        else:
            return '[' + str(production.get_left().get_head()['tag']) + ' ' + self.print_expansion(production.get_left()) + '][' + str(production.get_right().get_head()['tag']) + ' ' + self.print_expansion(production.get_right()) + ']'
       
    def check_duplicate(self, list, string): #this function assumes there is no ambiguity in the grammar, and that a string can only correspond to a set of rules
        for i in list:
            if i[0].get_text() == string:
                return True
        return False

    def print_rules(self):
        for r in self.rules:
            print(r)

#### Testing
Before going any further, I thought it was important to make sure that the functions contained in the grammar class were not going to fail. First, we can instantiate a basic grammar object, and pass some simple rules. The tags here are modified from this treebank: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

This is the first look at how a rule is structured within the nltk featStruct library. We go into more CFG details below, but now it's useful to analyze the tags contained within the parent, and its children.


In [94]:
test_cfg = Grammar()
test_cfg.add_rule(nltk.FeatStruct('[parent=[tag=S, terminal=false], child1=[tag=NP], child2=[tag=VP]]'))
test_cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=false], child1=[tag=DT], child2=[tag=NOM]]'))
test_cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true], child1=[tag=dog]]'))


The first important function is the find the LHS structure, or parent given a RHS. This is accomplished through the apply_rules() function

In [95]:
print(test_cfg.apply_rules('dog'))


[ tag      = 'NOM'  ]
[ terminal = 'true' ]


In [96]:
print(test_cfg.apply_rules('NP VP'))


[ tag      = 'S'     ]
[ terminal = 'false' ]


These both work as they are intended. The first recognizes that the word 'dog' in a nominal, and the second is able to work backwards and recognize the rule S -> NP VP. 

It is also going to be useful to work in the opposite direction. We may use a bottom-up approach to parse, but then later use a top down appoach for something else. By coding both of these functions from the start (and testing them), it will save time later on. To find the accompanying rule given a LHS string, we use the get_rules() function.

In [97]:
print(test_cfg.get_rules('S'))


[[child1=[tag='NP'], child2=[tag='VP'], parent=[tag='S', terminal='false']]]


In [98]:
print(test_cfg.get_rules('NOM'))

[[child1=[tag='dog'], parent=[tag='NOM', terminal='true']]]


Both tests pass. The next step will be building out the grammar to more closely resemble English.

#### Context-Free Grammars
A context-free grammars serve as a set of rules to construct patterns of strings. Their rules are soemetimes called productions, and they recursively describe how symbols can be expanded into new patterns. A context-free grammar contains:
- A group of terminal symbols, which are the characters of the alphabet that appear in the generataed strings
- A group of non-terminal symbols, which can be thought of as placeholders for patterns of terminal symbols 
- A set of productions, which are the rules of the grammar. They contain a left-hand-side and a right-hand side, and specify which symbols are included in the production
- A start symbol, which is the first non-terminal symbol from which strings are generated

For the purposes of this project, a CFG will be used to describe (in broad strokes) natural English Language. The grammar rules can be likened to the foundation of a theoretical Universal Grammar. To understand the fundamentals of CFGs a useful, I found this to be a useful document: https://www.cs.rochester.edu/~nelson/courses/csc_173/grammars/cfg.html.

A special form of CFGs is called Chomsky normal form. A CFG is in Chomsky normal form where the rules satisfy the one of the following conditions:
- A -> BC where A, B, and C are non-terminals and B and C are not the start terminal
- A -> a where A is non-terminal and a is terminal

Within this structure, the rules are stored with the feature structures unites [parent] -> [child1] [child2]

This structure is advantageous for both sentence generation and parsing. It allows us to structure the grammar as a binary tree, and exploit it for both its efficiency and simplicity. Here's another good source: https://courses.cs.washington.edu/courses/cse322/08au/lec14.pdf.

#### Part 1.A Reflection:
The main takeaway from this part would have been to conduct all of my planning and research prior to beginning. I found myself constantly backtracking on redundant code, or trying to fix the inadequacies in my grammar. The only important point was to practice good habits with writing object oriented code. This will be more pertinent a little bit later on, but doing so saves a lot of time when trying to access information. Rarely, even in school projects, do you construct a code base this large without any instruction. It was a great exercise in utilizing best practices in coding.

### B: Feature Structures

While CFGs are powerful, they cannot capture all of the grammatical information that we care about. A more elegant approach harnesses the power of feature structures. But what are feature structures?

Feature structures are sets of attribute/value pairs. They can hold non-grammatical inforamtion; however, they are commonly used to form agreement rules within a grammar. A feature structure can be represented as a directed acyclic graph (DAG), with the nodes corresponding to the variable values and the paths to the variable names.

Feature structures can be unified to combine the attribute/value pairs of two linguistic units. To get a better understanding of this, I recommend reading the nltk guide. It provides a lot of use case examples: https://www.nltk.org/book/ch09.html

There's a lot of agreement rules in English, and other languages have their own unique rules. However, a lot of these are fringe cases. A good example is how a language like relies heavily on gender - while English only relies on gender agreement for a few uses. The most notable is pronouns (SHE threw HER bag in the trunk). Take a step back from that and you'll see that pronoun's themselves present a difficult challenge for computers since tracing back their antecedent requires a complete understanding of semantic context.

With all of this in mind, I decided to focus on two important forms of agreement: Number and Tense. These were easily formalized and there weren't too many exceptions.

#### Number Agreement
Number agreement is concord in form (as singular, dual, or plural) between determiner-noun, pronoun-antecedent, and noun-verb. We will be using an NLTK feature structure grammar to declare these rules. Upone combining rules during generation or parsing, we will use the NLTK library to parse the new production, and see if it complies with the grammar.

#### Tense Consistency
Tense consistency is concord of verb tense across a clause. The simple forms are past, present, and future. 

#### Defining Rules
Here, the rules are defined that describe English. This may not look like much, but it includes all of the basic productions needed to produce English sentences, as well as the agreement rules that keep them together. Look at the comments for more details on each individual rule.

Below, we populate the terminal objects. The number and tense values are included in the agr feature structure. Eventually, we'll make direct comparisons between these structures.

In [99]:
cfg = Grammar()

cfg.add_rule(nltk.FeatStruct('[parent=[tag=S, terminal=false, agr=[num=?n]], child1=[tag=NP, agr=[num=?n]], child2=[tag=VP, agr=[num=?n]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=false, agr=[num=?n]], child1=[tag=DT, agr=[num=?n]], child2=[tag=NOM, agr=[num=?n]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=false, agr=[num=?n]], child1=[tag=DT, agr=[num=?n]], child2=[tag=NP, agr=[num=?n]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=false, agr=[num=pl]], child1=[tag=NP, agr=[]], child2=[tag=CCNP, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=false, agr=[num=pl]], child1=[tag=NOM, agr=[]], child2=[tag=NOM, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=false, agr=[num=?n]], child1=[tag=POS, agr=[]], child2=[tag=NOM, agr=[num=?n]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=false, agr=[num=?n]], child1=[tag=JJR, agr=[]], child2=[tag=NOM, agr=[num=?n]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=CCNP, terminal=false, agr=[]], child1=[tag=CC, agr=[]], child2=[tag=NOM, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=PP, terminal=false, agr=[]], child1=[tag=IN, agr=[]], child2=[tag=NP, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VB, agr=[num=?n, tense=?t]], child2=[tag=NP, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VB, agr=[num=?n, tense=?t]], child2=[tag=PP, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VP, agr=[num=?n, tense=?t]], child2=[tag=PP, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[]], child1=[tag=MD, agr=[]], child2=[tag=VP, agr=[tense=?t]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=AUX, agr=[tense=?t]], child2=[tag=VB, agr=[tense=?t]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=AUX, agr=[tense=?t]], child2=[tag=VP, agr=[tense=?t]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VP, agr=[tense=?t]], child2=[tag=CCVP, agr=[tense=?t]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VB, agr=[num=?n, tense=?t]], child2=[tag=RB, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VP, terminal=false, agr=[num=?n, tense=?t]], child1=[tag=VP, agr=[num=?n, tense=?t]], child2=[tag=RB, agr=[]]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=CCVP, terminal=false, agr=[tense=?t]], child1=[tag=CC, agr=[]], child2=[tag=VP, agr=[tense=?t]]]'))

#### Nominals, Pronouns, and Proper Nouns
For these, number agreement is critical.

In [100]:
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=sg]], child1=[tag=dog]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=sg]], child1=[tag=flight]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=pl]], child1=[tag=planes]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=sg]], child1=[tag=takeoff]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=sg]], child1=[tag=airport]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NOM, terminal=true, agr=[num=pl]], child1=[tag=children]]'))

#Proper Nouns and Pronouns are mapped directly to NP
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=I]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=me]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=Alaska]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=Seattle]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=Brian]]'))


cfg.add_rule(nltk.FeatStruct('[parent=[tag=NP, terminal=true, agr=[num=sg]], child1=[tag=I]]'))

#### Verbs, Verb Phrases, Modals, and Auxillaries
Verbs must account for number agreement, as well as tense.

In [101]:
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres], subcat=[empty=n, first=PP, rest=[empty=y]]], child1=[tag=fly]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=go]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=take]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=sg, tense=pres]], child1=[tag=leaves]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=hate]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=sg, tense=pres]], child1=[tag=is]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=are]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=shake]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=?n, tense=past]], child1=[tag=found]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=pl, tense=pres]], child1=[tag=ride]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[num=?n, tense=past]], child1=[tag=told]]'))

#AUX VERB - the production should only take the tense value of the auxillary verb
cfg.add_rule(nltk.FeatStruct('[parent=[tag=AUX, terminal=true, agr=[tense=pres]], child1=[tag=does]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=AUX, terminal=true, agr=[tense=past]], child1=[tag=have]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=AUX, terminal=true, agr=[tense=future]], child1=[tag=will]]'))

cfg.add_rule(nltk.FeatStruct('[parent=[tag=MD, terminal=true, agr=[tense=?t]], child1=[tag=can]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=MD, terminal=true, agr=[tense=?t]], child1=[tag=might]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=MD, terminal=true, agr=[tense=?t]], child1=[tag=could]]'))

#### Determiners and Conjunctions
Like nouns, determiners are primarily concerned with number agreement.

In [102]:
cfg.add_rule(nltk.FeatStruct('[parent=[tag=DT, terminal=true, agr=[num=?n]], child1=[tag=the]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=DT, terminal=true,agr=[num=sg]], child1=[tag=a]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=DT, terminal=true, agr=[num=pl]], child1=[tag=those]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=DT, terminal=true, agr=[num=sg]], child1=[tag=that]]'))

cfg.add_rule(nltk.FeatStruct('[parent=[tag=CC, terminal=true, agr=[]], child1=[tag=and]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=CC, terminal=true, agr=[]], child1=[tag=but]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=CC, terminal=true, agr=[]], child1=[tag=yet]]'))

#### Prepositions, Adjectives, Adverbs, and Possessives
The following inhertic features from the noun or verb they are pairing with.

In [103]:
cfg.add_rule(nltk.FeatStruct('[parent=[tag=IN, terminal=true, agr=[]], child1=[tag=from]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=IN, terminal=true, agr=[]], child1=[tag=up]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=IN, terminal=true, agr=[]], child1=[tag=off]]'))

cfg.add_rule(nltk.FeatStruct('[parent=[tag=RB, terminal=true, agr=[]], child1=[tag=quickly]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=RB, terminal=true, agr=[]], child1=[tag=wildly]]'))

cfg.add_rule(nltk.FeatStruct('[parent=[tag=JJR, terminal=true, agr=[]], child1=[tag=long]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=JJR, terminal=true, agr=[]], child1=[tag=tired]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=JJR, terminal=true, agr=[]], child1=[tag=big]]'))

#Superlative Adjectives
cfg.add_rule(nltk.FeatStruct('[parent=[tag=JJS, terminal=true, agr=[]], child1=[tag=longest]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=JJS, terminal=true, agr=[]], child1=[tag=worst]]'))

cfg.add_rule(nltk.FeatStruct('[parent=[tag=POS, terminal=true, agr=[]], child1=[tag=my]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=POS, terminal=true, agr=[]], child1=[tag=his]]'))

#### Random Expansion Testing

We have a random expansion function to make sure that we are able to generate sentences that are somewhat grammatrically sound. The flatten() function is used to clean up the output. The output has tendency to get distored since append() is used in conjunction iwht many recursive calls.

In [104]:
def flatten(items, seqtypes=(list, tuple)):
    for i, x in enumerate(items):
        while i < len(items) and isinstance(items[i], seqtypes):
            items[i:i+1] = items[i]
    return items

In [105]:
def expand_random(grammar, rule):
        
    print('----')
    print('expanding ' + rule + ':')
        
    sentence = []
 
    random_rule = random.choice(grammar.get_rules(rule))
            
    #probably can take out the is_terminal function
    if random_rule['parent']['terminal'] == 'true':
        print('found terminal node: ' + random_rule['child1']['tag'])
        return random_rule['child1']['tag']
                        
    if random_rule.has_key('child1'):
        child1_tag = random_rule['child1']['tag']
        sentence.append(expand_random(grammar, child1_tag))
            
    if random_rule.has_key('child2'):
        child2_tag = random_rule['child2']['tag']
        sentence.append(expand_random(grammar, child2_tag))
    
    return sentence

sentence = flatten(expand_random(cfg, 'S'))
print('\n'+' '.join([str(elem) for elem in sentence]))

----
expanding S:
----
expanding NP:
----
expanding DT:
found terminal node: those
----
expanding NOM:
----
expanding JJR:
found terminal node: tired
----
expanding NOM:
----
expanding JJR:
found terminal node: tired
----
expanding NOM:
found terminal node: airport
----
expanding VP:
----
expanding AUX:
found terminal node: does
----
expanding VP:
----
expanding AUX:
found terminal node: does
----
expanding VB:
found terminal node: shake

those tired tired airport does does shake


The result is gramatically sound according to the cfg, but is otherwise meaningless. It's still a fun and useful exercise to see what's possible from a few simple grammar rules and a top-down generator.

### Feature Unification:
Another thing to note about the random expansions above is that they lack any sort of agreement. In order to utilize the agreement structures in our grammar class, we need to harness the power of feature unification. Feature unification seeks to combine two nodes by looking at certain attribute value pairs. If the values match, then the matching value will be returned. Otherwise, it will return None.

#### Helper Classes
The following helper classes will be used in future tasks like parsing and phrase generation. We are going to declare them now to ensure that our agreement functions are compatible with those tasks. The following class is called Production. It serves as a richer linguistic unit that stores not only the part of speech, but pointers to children, the complete text of the linguistic unit.

In [106]:
class Production(object):
    
    head = None
    left = None
    right = None
    text = ''
    
    # Constructor which takes in the head of the rule, and two child rules
    def __init__(self, head, left, right, text):
        self.head = head
        self.left = left
        self.right = right
        self.text = text
    
    def get_head(self):
        return self.head
    
    def get_left(self):
        return self.left
    
    def get_right(self):
        return self.right
    
    def get_text(self):
        return self.text
    
    def get_attr(self):
        return self.attribute_label
    
#declare Cell later

#### Unification Function
The following unification function analyzes two children that the system is trying to merge. The first step is to understand the rule that is being used to merge the two, and obtain its parent. From there, the function will attempt to unify the 'agr' structure that is nested within each of the children, and return the result.

In [107]:
def unification(child1, child2, grammar):
            
    child1_fs_agr = child1.get_head()['agr']
    child2_fs_agr = child2.get_head()['agr']

    parent_fs = grammar.apply_rules(child1.get_head()['tag'] + ' ' + child2.get_head()['tag'])
    parent_fs_agr = parent_fs['agr']
    
    if child1_fs_agr is None or child2_fs_agr is None or parent_fs_agr is None:
        return None

    if parent_fs_agr.has_key('num'):
        
        if parent_fs_agr['num'] == 'sg' or parent_fs_agr['num'] == 'pl':
            
            child1_fs_agr['num'] = parent_fs_agr['num']
            child2_fs_agr['num'] = parent_fs_agr['num']
        
        try:
            return parent_fs_agr.unify(child1_fs_agr).unify(child2_fs_agr)
        except:
            return None
        
    elif parent_fs_agr.has_key('tense'):
        
        if child1.get_head()['tag'] == 'AUX' and child2_fs_agr.has_key('tense'):
            
            child2_fs_agr.pop('tense')
            
        try:
            return parent_fs_agr.unify(child1_fs_agr).unify(child2_fs_agr)
        except:
            return None    

    else:
        
        return parent_fs_agr

#### Testing
The goal here is to test the capabilities of number agreement, consistency, and the assumption of values when featured nodes are paired with unfeatured nodes (like an comparative adjective).

In [108]:
production_1 = Production(cfg.apply_rules('big'),None,None,'big')
production_2 = Production(cfg.apply_rules('dog'),None,None,'dog')
print(unification(production_1,production_2,cfg))

[ num = 'sg' ]


In [109]:
production_1 = Production(cfg.apply_rules('the'),None,None,'the')
production_2 = Production(cfg.apply_rules('children'),None,None,'children')
print(unification(production_1,production_2,cfg))

[ num = 'pl' ]


In [110]:
production_1 = Production(cfg.apply_rules('that'),None,None,'that')
production_2 = Production(cfg.apply_rules('children'),None,None,'children')
print(unification(production_1,production_2,cfg))

None


In [111]:
production_1 = Production(cfg.apply_rules('fly'),None,None,'fly')
production_2 = Production(cfg.apply_rules('quickly'),None,None,'quickly')
print(unification(production_1,production_2,cfg))

[ num   = 'pl'   ]
[ tense = 'pres' ]


The aboce examples showcase a few different outcomes. These can be either that the value is inferred from one node (such as Noun Phrase -> Nominal Adjective), the value is successfully unified from two objects (Noun Phrase -> Determiner Nominal), or the the values do not match (None is returned).

#### Part 1.B Reflection:

The was a lot to upack here, but something that I learned was with my code sometimes simpler was better. I learned that if i provided enough information in the actual grammar, that would make feature unification much simpler. 

The other takeaway, whichwas on the linguistic end was that I didn't always think about rules and agreement. They were things that I took for granted, and prior to this project, I actually would have had trouble articulating rules. That goes to show that when you are trying to formally encode information, you have to challenge assumptions and really do your homework.

### C: Verb Arguments
Another important part of our feature grammar is the ability to understand verb arguments. Verb argument structure is a complement to phrase structure grammar. It suggests that verbs belong to different classes according what events the verbs reference.

Events are occurances of the verb, and there are certain roles that play a part. These roles can be simple nouns serving as a receiving object, or more complex phrases. These supporting roles are sometimes called theta-roles. There are usually betweeen 0 and 3. For simplicity, we will look at the following classes of verbs:

#### Transitive:
Transitive verbs are characterized by the obligatory presence of two non-prepositional arguments: a
subject and a direct object. Transitive verbs assign accusative case to their direct object, while the
subject receives nominative case from the [+finite] specification of the Tense in the clause.

- John[AGENT] broke the window[PATIENT]
- John[AGENT] painted the door[THEME]
- John[AGENT] made dinner[FACITIVE]
- The wind[CAUSE] moves the grass[THEME]

#### DIntransitive:
Ditransitive verbs are very similar to transitive verbs but they have one more argument, which is
traditionally called indirect object. In both Italian and English it can be realized with a PP headed by
the preposition to/a. 

- John[ORIGIN] gave (a book)[THEME] (to Mary)[GOAL/RECEIVER]
- John sent (a letter)[THEME] (to New York)[GOAL/RECEIVER]

#### Intransitive:
There are verbs that take no argument. Among these verbs we find weather vebs, predicates that describe a situation such as the adjectives.

- It rains/ snows/ hails.
- It’s me. 

A good place for more information on this is: https://www.unive.it/media/allegato/download/Lingue/Materiale%20didattico%20Giusti/Lingua_Inglese1/Argument_structure1.pdf

#### How Feature Structure Represent This information
Our feature structures will include a sucategory field. Since the number of arguments is variable, we need to get creative with how we represent a list. This will look like the following:

In [112]:
ditrans = nltk.FeatStruct('[subcat=[empty=n, first=np, rest=[empty=n, first=pp, rest=[empty=y]]]]')
print(ditrans)

[          [ empty = 'n'                         ] ]
[          [ first = 'np'                        ] ]
[          [                                     ] ]
[ subcat = [         [ empty = 'n'             ] ] ]
[          [ rest  = [ first = 'pp'            ] ] ]
[          [         [                         ] ] ]
[          [         [ rest  = [ empty = 'y' ] ] ] ]


Here, we populate a few more examples:

In [113]:
#transitive
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[tense=past], subcat=[empty=n, first=NP, rest=[empty=y]]], child1=[tag=broke]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[tense=pres], subcat=[empty=n, first=PP, rest=[empty=y]]], child1=[tag=look]]'))
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[tense=past], subcat=[empty=n, first=PP, rest=[empty=y]]], child1=[tag=made]]'))

#ditransitive
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[tense=past], subcat=[empty=n, first=NP, rest=[empty=n, first=NP, rest=[empty=y]]]], child1=[tag=gave]]'))

#intransitive
cfg.add_rule(nltk.FeatStruct('[parent=[tag=VB, terminal=true, agr=[tense=pres], subcat=[empty=n, first=NP, rest=[empty=n, first=PP, rest=[empty=y]]]], child1=[tag=rains]]'))


In [114]:
#should only get called if the first argument is a vb or vp and has a subcat field - returns the new subcat fs
def verb_argument(child1, child2, grammar):
            
    verb_subcat = child1.get_head()['subcat']
    object_type = child2.get_head()['tag']
    
    if verb_subcat['empty'] == 'y':
        
        if object_type == 'NOM' or object_type == 'NP' or object_type == 'PP':
            return None
        else:
            return verb_subcat
        
    else:
        
        if verb_subcat['first'] == object_type:
            return verb_subcat['rest']
        else:
            return None

#### Testing/Demo
In this simple example: the verb is gave, which is ditransitive. it is expected NP for the first argument. However, the unification fails since it is passed an adverb

In [115]:
production_1 = Production(cfg.apply_rules('gave'),None,None,'gave')
production_2 = Production(cfg.apply_rules('wildly'),None,None,'wildly')

print(production_1.get_head()['subcat'])
print('----')
print(verb_argument(production_1, production_2, cfg))

[ empty = 'n'                         ]
[ first = 'NP'                        ]
[                                     ]
[         [ empty = 'n'             ] ]
[ rest  = [ first = 'NP'            ] ]
[         [                         ] ]
[         [ rest  = [ empty = 'y' ] ] ]
----
None


Instead, if the verb is given a NP - in this case a proper noun - it returns the subcat feature structure associated with the resulting unification.

In [116]:
production_1 = Production(cfg.apply_rules('gave'),None,None,'gave')
production_2 = Production(cfg.apply_rules('Brian'),None,None,'Brian')

print(production_1.get_head()['subcat'])
print('----')
print(verb_argument(production_1, production_2, cfg))

[ empty = 'n'                         ]
[ first = 'NP'                        ]
[                                     ]
[         [ empty = 'n'             ] ]
[ rest  = [ first = 'NP'            ] ]
[         [                         ] ]
[         [ rest  = [ empty = 'y' ] ] ]
----
[ empty = 'n'             ]
[ first = 'NP'            ]
[                         ]
[ rest  = [ empty = 'y' ] ]


#### Part 1.C Reflection:

I think the biggest takeaway from part c was just more carryover from part 2. Looking at predicate argument structure showed me that there are sometimes multiple solution and clever workarounds when working with both linguistics and computer science.

## Part 2: CFG Parsing
In order to round out the program, we are going to test the CFG rules in a reverse manner: by parsing setences using the CYK parsing algorithm (Cocke–Younger–Kasami). This algorithm is designed for use specifically with CFGs and shows if sentences are accepted by the grammar. It uses a bottom-up appraoch with dynamic programming.

Here is a very useful tool to visualize the parse table: https://www.xarg.org/tools/cyk-algorithm/

#### Supporting Objects Structures
We need some addition helper objects to aid in parsing (and eventually search-based sentence generation). Notably, As the setence parses, there is some expected ambiguity. To account for this, we need to create a cell object that holds the multiple production rules that are an option for a given token or set of tokens.

In [117]:
class Cell(object):
    
    #list of production rules
    productions = []
    
    #basic constructor with an optional parameter productions
    def __init__(self, productions=None):
        if productions is None:
            self.productions = []
        else:
            self.productions = productions
            
    def add_production(self, result, p1, p2, text):
        new_production = Production(result, p1, p2, text)
        self.productions.append(new_production)
    
    def set_productions(self, p):
        self.productions = p
    
    def get_results(self):
        results = []
        for p in self.productions:
            results.append(p.get_head()['tag'])
            
        return results
    
    def get_rules(self):       
        return self.productions

#### Parsing
The CYK algorithm parses using a dynamic table to help the possible derivations, and constructing a viable tree from the bottom up. We can implement this with a class that stores cell objects. Within the class, we can also put our parse function, that parses an input string.

In [118]:
class Parse_Table(object):
    
    def __init__(self, grammar):
        self.grammar = grammar
        self.parse_table = None
        self.length = 0
    
    #Print the CYK parse trable for the last sentence that have been parsed - this requires the tabulate library
    def print_parse_table(self):      
        lines = [] 
        
        for row in reversed(self.parse_table):
            l = []
            for cell in row:
                l.append(cell.get_results())
            lines.append(l)
        
        lines.append(self.tokens)
        print(tabulate(lines))
        
    #Returns a list containing the parent of the possible trees that we can generate for the last sentence that have been parsed
    def get_trees(self):
        return self.parse_table[self.length-1][0].productions
    
    #Parse a sentence (string) with the CYK algorithm   
    def parse(self, sentence):
        self.number_of_trees = 0
        self.tokens = sentence.split()
        self.length = len(self.tokens)
        
        if self.length < 1:
            print('Error: unable to parse')
            return
        
        #list comprehension to declare an empty parse table
        self.parse_table = [ [Cell() for x in range(self.length - y)] for y in range(self.length) ]
        
        
        for x, t in enumerate(self.tokens):
            
            r = self.grammar.apply_rules(t)
            
            if r == None:
                raise ValueError("The word " + str(t) + " is not in the grammar")
            else:
                self.parse_table[0][x].add_production(r, None, None, str(t))        
                
        
        for l in range(2,self.length+1):
            for s in range(1,self.length-l+2):
                for p in range(1,l-1+1):
                    
                    t1 = self.parse_table[p-1][s-1].get_rules()
                    t2 = self.parse_table[l-p-1][s+p-1].get_rules()
                            
                    for a in t1:
                        for b in t2:
                            
                            r = self.grammar.apply_rules(str(a.get_head()['tag']) + " " + str(b.get_head()['tag']))
                            
                            if r is not None:
                                
                                if (a.get_head()['tag'] == 'VP' or a.get_head()['tag'] == 'VB') and a.get_head().has_key('subcat'):
                                    return_subcat = verb_argument(a, b, self.grammar)
                                    if return_subcat is None:
                                        continue
                                    else:
                                        print('--------')
                                        print('Unified verb argument: ['+str(a.get_head()['tag'])+' '+a.get_text()+'] ['+str(b.get_head()['tag'])+' '+b.get_text()+']')
                                        print(return_subcat)
                                        print('--------')
                                        r['subcat'] = return_subcat
                    
                                return_agr = unification(a,b,self.grammar)
                                if return_agr is None:
                                    print(str(a.get_head()['agr']))
                                    print(str(b.get_head()['agr']))
                                    print('Error number/tense agreement: ['+str(a.get_head()['tag'])+' '+a.get_text()+'] ['+str(b.get_head()['tag'])+' '+b.get_text()+']')
                                    continue
                                else:
                                    print('--------')
                                    print('Unified number/tense agreement: ['+str(a.get_head()['tag'])+' '+a.get_text()+'] ['+str(b.get_head()['tag'])+' '+b.get_text()+']')
                                    print(return_agr)
                                    print('--------')
                                    r['agr'] = return_agr
                                    
                                print('Applied Rule: ' + r['tag'] + '[' + str(l) + ',' + str(s) + ']' + ' --> ' + str(a.get_head()['tag']) + '[' + str(p) + ',' + str(s) + ']' + ' ' + str(b.get_head()['tag'])+ '[' + str(l-p) + ',' + str(s+p) + ']')  
                                self.parse_table[l-1][s-1].add_production(r,a,b,a.get_text() + ' ' + b.get_text())
                               
        self.number_of_trees = len(self.parse_table[self.length-1][0].get_results())
        print("---------------------------------------")
        print('Number of possible trees: ' + str(self.number_of_trees))

#### Testing
The folling example should test multiple aspects of the grammar. First, it will test the usage of a modal verb, adding complexity to basic structure. Also it will test number agreement beetween 'I' and 'fly'.



In [119]:
pt = Parse_Table(cfg)
pt.parse('I might fly from Alaska')
pt.print_parse_table()
trees = pt.get_trees()

--------
Unified number/tense agreement: [IN from] [NP Alaska]
[]
--------
Applied Rule: PP[2,4] --> IN[1,4] NP[1,5]
--------
Unified verb argument: [VB fly] [PP from Alaska]
[ empty = 'y' ]
--------
--------
Unified number/tense agreement: [VB fly] [PP from Alaska]
[ num   = 'pl'   ]
[ tense = 'pres' ]
--------
Applied Rule: VP[3,3] --> VB[1,3] PP[2,4]
--------
Unified number/tense agreement: [MD might] [VP fly from Alaska]
[]
--------
Applied Rule: VP[4,2] --> MD[1,2] VP[3,3]
--------
Unified number/tense agreement: [NP I] [VP might fly from Alaska]
[ num = 'sg' ]
--------
Applied Rule: S[5,1] --> NP[1,1] VP[4,2]
---------------------------------------
Number of possible trees: 1
------  ------  ------  ------  ------
['S']
[]      ['VP']
[]      []      ['VP']
[]      []      []      ['PP']
['NP']  ['MD']  ['VB']  ['IN']  ['NP']
I       might   fly     from    Alaska
------  ------  ------  ------  ------


#### Part 2 Reflection:

Pary 2 was great since I felt like I was incorporating so many different parts of my undergraduate coursework into 1 algorith. It was logic, dynamic programming, and parsing concepts all packed into 1. I think being able to pass in testing sentences also really helped to evaluate the efficacy of my grammar rules.

## Part 3: Search-Based NLG
Now, we will begin generating semantically-important text on top of the context-free grammar. The alogorithm that is most up to task is best-first search. This is because search allows you to explore multiple different paths. These paths begin in a queue and are popped so that neighboring options can be viewed. It supports the usage of heuristic functions, which evaluates which path is most promising. I used this guide to trace the algorithm: https://www.geeksforgeeks.org/best-first-search-informed-search/

Initially, we will be generating ALL possible phrases given a list of entries. However, later on we will experiment with different heuristics to improve the output of the search algorithm.

This particiular instance of the search function is quite simple. It begins with a single list of productions. When it finds a new way to join two nodes together, it pops the two nodes from the list, joins them and adds it back to the queue. It continues to do this, effectively creating branches of potential tree derviations.

Note: check_duplicate() is intended to prevent redundancy in the queue

In [120]:
def search(start, grammar):
        
        #stores complete sentences (sentences that are the correct length and have 'S' at the head)
        complete_sentences = []
        
        #queue that holds lists of productions
        q = []
        #first list is added before the loop
        q.append(start)
        
        while len(q) > 0:
            #item at first index is popped, the algorithm still assumes CNF and attempts to nodes that can be joined
            curr_list = q.pop(0) 
            for i in range(len(curr_list)):
                for j in range(len(curr_list)):
                    
                    #checks to make sure the node did not match with itself
                    if curr_list[j] != curr_list[i]:
                        
                        if(curr_list[i].get_head() is None or curr_list[j].get_head() is None):
                            continue
                                                
                        #finds the rule that joins the two nodoes
                        if grammar.apply_rules(str(curr_list[i].get_head()['tag']) + ' ' + str(curr_list[j].get_head()['tag'])) != None:
                            first_node = curr_list[i]
                            first_index = i
                            second_node = curr_list[j] 
                            second_index = j
                        elif grammar.apply_rules(str(curr_list[j].get_head()['tag']) + ' ' + str(curr_list[i].get_head()['tag'])) != None:
                            first_node = curr_list[j]
                            first_index = j
                            second_node = curr_list[i]    
                            second_index = i
                        else:
                            continue
                        
                        temp_head = grammar.apply_rules(str(first_node.get_head()['tag']) + ' ' + str(second_node.get_head()['tag']))

                        if (first_node.get_head()['tag'] == 'VP' or first_node.get_head()['tag'] == 'VB') and first_node.get_head().has_key('subcat'):
                            return_subcat = verb_argument(first_node, second_node, grammar)
                            if return_subcat is None:
                                continue
                            else:
                                print('--------')
                                print('Unified verb argument: ['+str(first_node.get_head()['tag'])+' '+first_node.get_text()+'] ['+str(second_node.get_head()['tag'])+' '+second_node.get_text()+']')
                                print(return_subcat)
                                print('--------')
                                temp_head['subcat'] = return_subcat
                    
                        return_agr = unification(first_node,second_node,grammar)
                        if return_agr is None:
                            print(str(first_node.get_head()['agr']))
                            print(str(second_node.get_head()['agr']))
                            print('Error number/tense agreement: ['+str(first_node.get_head()['tag'])+' '+first_node.get_text()+'] ['+str(second_node.get_head()['tag'])+' '+second_node.get_text()+']')
                            continue
                        else:
                            print('--------')
                            print('Unified number/tense agreement: ['+str(first_node.get_head()['tag'])+' '+first_node.get_text()+'] ['+str(second_node.get_head()['tag'])+' '+second_node.get_text()+']')
                            print(return_agr)
                            print('--------')
                            temp_head['agr'] = return_agr
                        
                        #makes a copy of the list popped from q -> this is so the list can be modified and pushed back
                        temp_list = curr_list.copy()
                            
                        #to prevent the wrong item from getting popped if the list shrinks after the first
                        if first_index < second_index:
                            temp_node_1 = temp_list.pop(first_index)
                            temp_node_2 = temp_list.pop(second_index-1)
                        else:
                            temp_node_1 = temp_list.pop(first_index)
                            temp_node_2 = temp_list.pop(second_index)
                                
                        #creates a new node and modifies it to the copied list
                        temp_string = temp_node_1.get_text() + ' ' + temp_node_2.get_text()
                        temp_list.append(Production(temp_head, temp_node_1, temp_node_2, temp_string))
                            
                        #tries to prevent duplicate
                        if temp_list not in q:
                                
                            #if the node meets conditions, added to complete sentences
                            if temp_head['tag'] == 'S' and len(temp_string.split(' ')) == len(start) and not grammar.check_duplicate(complete_sentences, temp_string):
                                complete_sentences.append(temp_list)
                                
                            q.append(temp_list)
                                    
        #prints complete sentences along with corresponding tags
        for i in complete_sentences:
            print(i[0].get_text())
            print('[' + str(i[0].get_head()['tag']) + ' ' + grammar.print_expansion(i[0]) + ']')
            print('-------------------------')

#### Testing
Test the following list of inputs. All of the words are already part of the grammar.

In [121]:
inputs = ['might','Alaska','I','from','fly']
productions = []
for i in inputs:
    productions.append(Production(cfg.apply_rules(i),None,None,i))
search(productions, cfg)

--------
Unified number/tense agreement: [IN from] [NP Alaska]
[]
--------
--------
Unified number/tense agreement: [IN from] [NP I]
[]
--------
--------
Unified number/tense agreement: [IN from] [NP Alaska]
[]
--------
--------
Unified number/tense agreement: [IN from] [NP I]
[]
--------
--------
Unified verb argument: [VB fly] [PP from Alaska]
[ empty = 'y' ]
--------
--------
Unified number/tense agreement: [VB fly] [PP from Alaska]
[ num   = 'pl'   ]
[ tense = 'pres' ]
--------
--------
Unified verb argument: [VB fly] [PP from Alaska]
[ empty = 'y' ]
--------
--------
Unified number/tense agreement: [VB fly] [PP from Alaska]
[ num   = 'pl'   ]
[ tense = 'pres' ]
--------
--------
Unified verb argument: [VB fly] [PP from I]
[ empty = 'y' ]
--------
--------
Unified number/tense agreement: [VB fly] [PP from I]
[ num   = 'pl'   ]
[ tense = 'pres' ]
--------
--------
Unified verb argument: [VB fly] [PP from I]
[ empty = 'y' ]
--------
--------
Unified number/tense agreement: [VB fly] [

Here, we can analyze the unifications steps taken during search. The printed statements are the strucutres that are returned to each parent. The tagged sentences at the bottom are both valid setences according to the context free grammar. Also note that they were able to parse this sentence earlier. That signifies that our grammar works as a medium for both parsing and generation tasks.

### Part 4 Reflection

This section was useful since it really showed the round trip program. I liked seeing how everything built on top of the simple grammar class. I also enjoyed making engineering decisions to cut down on overhead in this section - namely, trying to keep duplicates out of the search queue.

## Project Summary 
I am extremely satisfied with the result and how the notebook seemed to evolve from my original idea. I think the most interesting part is how things seemed to fit together like a puzzle. Aspects of language that previously seemed arbitrarily constructed could actually be codified into rules. More interesting than that even was how parsing and generation took very similar approaches to interpreting nodes and building out trees. I think any individual, even someone with little to no knowledge of the area, would appraoch language with a different perspective after gaining a basic understanding of these concepts.

This notebook does a very good job with how language can be structured within code. There is no doubt that this code could become more and more complex with the structure. This comes from more grammar rules, edge cases, and heuristics to help tackle structural ambiguities. However, a bigger question to tackle is how to represent meaning. Is it possible to approach the idea of semantics with the same formality, or would one have to turn to a more statistical approach. This is the next think I would explore, to begin placing meaning contraints on generations, and even analyze the intersection of language and logic to solve simple problems.

Additionally, one of the original goals of this paper was to make the example-based. That comes with some kind of learning mechanism, to take the patterns from above and extrapolate them to gain an understanding of new rules. There were many facets to this problem, ranging from different learning algorithms, to how to tag inputs and induce their linguistic information. However, one of my goals would still be to keep the training process as lightweight as possible. I am still very much interested in exploring a nativist appraoch to grammar, and test poverty of stimulus simulation. Therfore, the central idea to tackle would be: how to provide a rigid foundation of language so that the the system could acquire language from as little information as possible.